# **Project Final Report**

Selima Yang, Mia Chow, Daniel Zhang

### (1) Introduction:

The project that we are working on surrounds the prediction of the usage of a video game research server. We are provided two datasets as a supplement to answer any predictive questions we came up with. This dataset is provided by the Pacific Laboratory for Artificial Intelligence, and they aim to understand in more detail the actions of players in the video game, Minecraft. 

Question: How does the amount of hours a player spend on the server, and the number of sessions predict gender? 
- The number of sessions and the amount of hours are the predictor variables
- The gender of players is the response variable of interest

### (2) Methods & Results

We first load the data using HTML form, which allows us to see the dataset in the CSV format. Then, we select the variables in question: hashedEmail, played_hours, and gender from the players.csv dataset, and hashedEmail, sessioncount from the second dataset. We join the two datasets together. 



is it still loading

https://github.com/mia6c/dsci-100-group-project.git